In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [4]:
# Load .env for API keys, etc.
load_dotenv()
client = OpenAI()

In [33]:
def get_enriched_prompt(predicted_info, max_distance, max_emission, max_emission_grams, predicted_mode):
    template = f"""
You are a sustainability logistics planner. Based on the predicted route information below, generate a full multi-segment route that:

- Stays within a total distance of {max_distance} km.
- Stays under a total CO₂ emission of {max_emission} kg (i.e., {max_emission_grams} grams).
- Primarily uses the predicted mode: {predicted_mode}, but you may use other modes to optimize for emissions or realism.
- Choose realistic transport paths (city-to-city freight rail, ports, road access).
- Prefer sustainable alternatives: Cargo Train, Container Ship, Electric Van.
- Limit to maximum 3 hopes

Emission rates:
- Air Freight: 600 g/km
- Container Ship: 10 g/km
- Cargo Train: 50 g/km
- Heavy Truck: 500 g/km
- Medium Truck: 300 g/km
- Small Van: 180 g/km
- Electric Van: 40 g/km

Each segment must include: `from`, `to`, `country`, `mode`, `vehicle`, `distance_km`, `emission_g`

Output JSON:
{{
  "route": [{{...}}],
  "total_distance_km": int,
  "total_emission_g": int
}}

### Predicted Info:
{predicted_info}
"""
    return template

In [13]:
from pydantic import BaseModel

class Route(BaseModel):
    start_from: str
    to: str
    country: str
    mode: str
    vehicle: str
    distance_km: int
    emission_g: int

class Routes(BaseModel):
    routes: list[Route]


In [34]:
import json
def generate_route(predicted_info, model="gpt-4o", temperature=0.7, n=3):
    routes = []
    # predicted_info=json.dumps(predicted_info, indent=2)
    max_distance=predicted_info["distance"]
    max_emission=predicted_info["emission"]
    max_emission_grams=int(float(predicted_info["emission"]) * 1000)
    predicted_mode=predicted_info["transportation_mode"]
    enriched_prompt = get_enriched_prompt(predicted_info, max_distance, max_emission, max_emission_grams, predicted_mode)
    for _ in range(n):
        response = client.chat.completions.parse(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful sustainability logistics planner."},
                {"role": "user", "content": enriched_prompt}
            ],
            response_format=Routes,
            temperature=temperature,
        )
        try:
            #route_data = json.loads(response)
            result = response.choices[0].message.content
            routes.append(result)
        except Exception as e:
            print(f"⚠️ Failed to parse JSON: {e}")
    return routes


In [35]:
predicted_info = {
    'source': 'Madrid',
    'source_country': 'Spain',
    'destination': 'Istanbul',
    'destination_country': 'Turkey',
    'transportation_mode': 'Cargo Train',
    'distance': 2855,
    'emission': 296
}

candidate_routes = generate_route(predicted_info, n=3)

In [ ]:
def validate_route(route, max_distance_km, max_emission_g):
    total_distance = route["total_distance_km"]
    total_emission = route["total_emission_g"]

    return {
        "distance_valid": total_distance <= max_distance_km,
        "emission_valid": total_emission <= max_emission_g,
        "status": "valid" if total_distance <= max_distance_km and total_emission <= max_emission_g else "invalid"
    }


In [36]:
import json

raw_outputs = candidate_routes
# Parse and enrich with total distance/emission
parsed_routes = []
for raw in raw_outputs:
    data = json.loads(raw)
    segments = data["routes"]
    total_distance = sum(seg["distance_km"] for seg in segments)
    total_emission = sum(seg["emission_g"] for seg in segments)
    parsed_routes.append({
        "route": segments,
        "total_distance_km": total_distance,
        "total_emission_g": total_emission
    })

# Rank by emission
top_routes = sorted(parsed_routes, key=lambda r: r["total_emission_g"])[:3]

# Print results
print(f"\n Source: {predicted_info['source']} ")
print(f"Destination: {predicted_info['destination']} ")
print(f"\n📦 Predicted Max Distance: {predicted_info['distance']} km")
print(f"♻️  Predicted Max Emission: {predicted_info['emission']} kg CO₂")

print("\nTop 3 generated routes that meet the predicted maximum distance and emission constraints.\n")

for i, route in enumerate(top_routes, 1):
    print(f"🚛 Top Route {i}:")
    print(f"Total Distance: {route['total_distance_km']} km")
    print(f"Total Emission: {route['total_emission_g']/1000} kg CO₂")
    print("Segments:")
    for seg in route["route"]:
        print(f"  {seg['start_from']} → {seg['to']} | {seg['mode']} | {seg['distance_km']} km | {seg['emission_g']/1000} kg")



 Source: Madrid 
Destination: Istanbul 

📦 Predicted Max Distance: 2855 km
♻️  Predicted Max Emission: 296 kg CO₂

Top 3 generated routes that meet the predicted maximum distance and emission constraints.

🚛 Top Route 1:
Total Distance: 2825 km
Total Emission: 101.97 kg CO₂
Segments:
  Madrid → Barcelona | Cargo Train | 621 km | 31.05 kg
  Barcelona → Genoa | Container Ship | 982 km | 9.82 kg
  Genoa → Istanbul | Cargo Train | 1222 km | 61.1 kg
🚛 Top Route 2:
Total Distance: 2820 km
Total Emission: 105.0 kg CO₂
Segments:
  Madrid → Barcelona | Rail | 620 km | 31.0 kg
  Barcelona → Genoa | Sea | 900 km | 9.0 kg
  Genoa → Istanbul | Rail | 1300 km | 65.0 kg
🚛 Top Route 3:
Total Distance: 2855 km
Total Emission: 112.75 kg CO₂
Segments:
  Madrid → Barcelona | Cargo Train | 620 km | 31.0 kg
  Barcelona → Genoa | Container Ship | 750 km | 7.5 kg
  Genoa → Istanbul | Cargo Train | 1485 km | 74.25 kg


In [ ]:
predicted_max_distance_km = int(predicted_info["distance"])
predicted_max_emission_g = int(float(predicted_info["emission"]) * 1000)

for i, route in enumerate(top_routes, 1):
    result = validate_route(route, predicted_max_distance_km, predicted_max_emission_g)

    print(f"\n🚛 Route {i}:")
    print(f"  Total Distance: {route['total_distance_km']} km")
    print(f"  Total Emission: {route['total_emission_g']} g CO₂")
    print(f"  ✅ Distance Valid: {result['distance_valid']}")
    print(f"  ✅ Emission Valid: {result['emission_valid']}")
    print(f"  🧾 Overall Status: {result['status']}")

    for seg in route["route"]:
        print(f"    - {seg['start_from']} → {seg['to']} | {seg['mode']} | {seg['distance_km']} km | {seg['emission_g']} g")
